In [1]:
import pandas as pd              # For dataframes
import matplotlib.pyplot as plt  # For plotting data
from apyori import apriori       # For Apriori algorithm
import ast

In [2]:
#pip install apyori

In [3]:
data = pd.read_excel('final_data.xlsx')
data

,NO.TRANSAKSI,DATE,DESCRIPTION,QTY
0,2301011000001,2023-01-01,MADU NSTR SUPER 650ML.,1
1,2301011000001,2023-01-01,STELLA MTC F/C 225ML.REF /12,1
2,2301011000001,2023-01-01,GELAS KOPI BOLA@50,2
3,2301011000001,2023-01-01,MIKA BX IV,3
4,2301011000002,2023-01-01,LAGIE GOLD CM 75G.W/30 LG07507,2
...,...,...,...,...
21929,2312311020099,2023-12-31,INDOMIE SOTO MIE/40 SM,1
21930,2312311020099,2023-12-31,INDOMIE KARI AYAM/40 IKA,1
21931,2312311020099,2023-12-31,INDOMIE KALDU AYAM/40 KA75,3
21932,2312311020099,2023-12-31,SEDAAP MIE KOREAN SPCY40-20234,1


In [4]:
# Menggabungkan kolom DESCRIPTION berdasarkan NO.TRANSAKSI
pd.set_option('display.max_colwidth', None)
grouped_data = data.groupby('NO.TRANSAKSI').agg({
    'DESCRIPTION': lambda x: list(x),  # Menggabungkan DESCRIPTION menjadi list
}).reset_index()

grouped_data

,NO.TRANSAKSI,DESCRIPTION
0,2301011000001,"[MADU NSTR SUPER 650ML., STELLA MTC F/C 225ML.REF /12, GELAS KOPI BOLA@50, MIKA BX IV]"
1,2301011000002,"[LAGIE GOLD CM 75G.W/30 LG07507, NABATI RCCO W.125G/24 300145]"
2,2301011000003,"[SANIA T.TERIGU 1KG. /10, ZEN BARSOAP @4 TEA TREE /24, HAPPY SWEET GULA 1KG/24, MIRATONE CLR SCT 16ML 2BL, SASA- 100G/10PER 112000028, BERAS AGUNG 5KG. /8]"
3,2301011000004,"[SWALLOW NAPTH 100G S-125, SEDAAP MIE KOREAN SP. 40-20261, KINGS/F SARD.SMBL 155G/50 .415, TRESEMME KERATIN 170/18.558380, SAMYANG HOT CHICK.SPICY BOW/16, SEDAAP MIE SINGAPORE SP.40-310, SEDAAP MIE WH.CURRY 40-20134, TAS SPUNBOND TULIP M]"
4,2301011000005,[WALLS MAGNUM V.ALMD/24 .70743]
...,...,...
3549,2312311020095,"[SANIA T.TERIGU 1KG. /10, BONTEH MELATI 330ML/24 D601XX]"
3550,2312311020096,"[PIXY DEO'S WOODY 34G, PEPSODENT PG WHITE 225G/36.235, LIFEBUOY SOAP WHT 4X100G. /24, MILKIDO COKELAT 200ML, KANZLER SINGLE SOSIS ORI 60G.]"
3551,2312311020097,"[BAYFRESH REED D.SAKURA/6 BRD02, MONTISS TISSU 250S R1/48.20250, AUTAN ACT&PROT.6X6ML.ALL NIGHT, BUAH SEMANGKA POTONG, TAS SPUNBOND TULIP M]"
3552,2312311020098,"[GARUDA ROSTA 100G/30 ORBM, LE MINERALE 1500ML/12 351506, LAURIER RLX NIG WGAT.40CM@16/9, KODOMO BABY BLUE 50'S 24-10710, NICE SOFT PACK@200/20 .1345, BIORE BF WHT SCRUB 250ML. /24, ZINC SHP 70ML HAIR/F 24-10404, LAURIER SS GUARD 22CM@14'S /16, TESSA TP-11 @50 TRVL.PACK /80, VAPE ONE PUSH 15HR.ORG /12, RINSO MOLTO ESSENCE 38G/126, VAPE ONE PUSH 15HR.ORG /12, ZINC SHP 70ML HAIR/F 24-10404, ENERGEN 5X32G VAN /32 RCG, GAMI GORENG SPECIAL/40 857940, HUKI COTT.B CI0009 50'S ZAK/12, HUKI COT/BUD CI0082 150'S, SEAGUL SG-535 TOILET BALL@3, BAGUS NAPTH 35G. W3603, SWALLOW NAPTH 35G S-111, SWAN NAPTH 35G S-200]"


In [5]:
transactions = grouped_data['DESCRIPTION'].tolist()
transactions

[['MADU NSTR SUPER 650ML.',
  'STELLA MTC F/C 225ML.REF   /12',
  'GELAS KOPI BOLA@50',
  'MIKA BX IV'],
 ['LAGIE GOLD CM 75G.W/30 LG07507', 'NABATI RCCO W.125G/24   300145'],
 ['SANIA T.TERIGU 1KG.        /10',
  'ZEN BARSOAP @4 TEA TREE    /24',
  'HAPPY SWEET GULA 1KG/24',
  'MIRATONE CLR SCT 16ML 2BL',
  'SASA- 100G/10PER     112000028',
  'BERAS AGUNG  5KG.          /8'],
 ['SWALLOW NAPTH 100G       S-125',
  'SEDAAP MIE KOREAN SP. 40-20261',
  'KINGS/F SARD.SMBL 155G/50 .415',
  'TRESEMME KERATIN 170/18.558380',
  'SAMYANG HOT CHICK.SPICY BOW/16',
  'SEDAAP MIE SINGAPORE SP.40-310',
  'SEDAAP MIE WH.CURRY   40-20134',
  'TAS SPUNBOND TULIP M'],
 ['WALLS MAGNUM V.ALMD/24  .70743'],
 ['NYU HAIR CLR NTRL BLEACH   /24', 'NYU HAIR CLR FLAMINGO ORG  / 2'],
 ['OREO SOFT CAKE 16G.      12/12'],
 ['SAKINAH  600ML', 'AQISAH SARI ASEM JAWA 500ML'],
 ['SASA SANTAN BBK.20G/120 ..0293',
  'NEXTAR NASTAR P.90G/30  303973',
  'ANTANGIN ANAK 10ML/5/60 010063',
  'ROMA MALKIST C.90G/30   310390',


In [6]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,3M MASKER RESPIRATOR@1/20.3283,ABC ALK A2-2 LR06-2B+1B 5078,ABC ALK LR03 4B+2B/72 2184,ABC ALK LR06 2B MP/12 484,ABC ALK LR06 4B+2R9/72 2183,ABC ALKALINE A2-4 LR6/72 4149,ABC ALKALINE A3-2 LR03 5079,ABC BAT BIRU CZ R20/12 456,ABC BAT BIRU R14-2/72 457,ABC BAT BIRU R6-12/24 480,...,ZWITSAL BB MYK TLON 250ML. /12,ZWITSAL BB NAT M.TELON 60ML,ZWITSAL COL FLO KISS 100/24.32,ZWITSAL EAU DE TOILETTE 100/12,ZWITSAL H/LOT ALOE V.200ML /18,ZWITSAL HAIR LOT ALMD.100ML/24,ZWITSAL SHP ALOE 100ML /36,ZWITSAL SOAP 4X70G/36 67557840,ZWITSAL SOAP CLSS.70G/144.7872,ZWITSAL SOAP R/HONEY 70G. /144
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3549,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3550,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3551,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3552,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.01)

,support,itemsets
0,0.011818,(370)
1,0.010411,(407)
2,0.018289,(409)
3,0.010974,(938)
4,0.010411,(941)
5,0.012380,(2295)
6,0.011818,(2297)
7,0.010692,(2301)
8,0.014631,(2302)
9,0.014631,(2305)


In [12]:
apriori(df, min_support=0.01, use_colnames=True)

,support,itemsets
0,0.011818,(BEAR BRAND 189ML/30 8109004)
1,0.010411,(BENG-BENG REG.25G/136 410175)
2,0.018289,(BERAS AGUNG 5KG. /8)
3,0.010974,(CLUB 1500ML./12 3015)
4,0.010411,(CLUB 600ML./24 4019)
5,0.012380,(INDOMIE AYAM BAWANG/40 AB)
6,0.011818,(INDOMIE GORENG ACEH/40 IMGA)
7,0.010692,(INDOMIE GRG JMBO A.PGG/24 GAPJ)
8,0.014631,(INDOMIE GRG JUMBO/24 GSSJ)
9,0.014631,(INDOMIE GRG RENDANG/40 GRS)


In [14]:
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.011818,(BEAR BRAND 189ML/30 8109004),1
1,0.010411,(BENG-BENG REG.25G/136 410175),1
2,0.018289,(BERAS AGUNG 5KG. /8),1
3,0.010974,(CLUB 1500ML./12 3015),1
4,0.010411,(CLUB 600ML./24 4019),1
5,0.012380,(INDOMIE AYAM BAWANG/40 AB),1
6,0.011818,(INDOMIE GORENG ACEH/40 IMGA),1
7,0.010692,(INDOMIE GRG JMBO A.PGG/24 GAPJ),1
8,0.014631,(INDOMIE GRG JUMBO/24 GSSJ),1
9,0.014631,(INDOMIE GRG RENDANG/40 GRS),1


In [ ]:
frequent_itemsets[(frequent_itemsets['support'] >= 0.01) ]

,support,itemsets,length
0,0.011818,(BEAR BRAND 189ML/30 8109004),1
1,0.010411,(BENG-BENG REG.25G/136 410175),1
2,0.018289,(BERAS AGUNG 5KG. /8),1
3,0.010974,(CLUB 1500ML./12 3015),1
4,0.010411,(CLUB 600ML./24 4019),1
5,0.012380,(INDOMIE AYAM BAWANG/40 AB),1
6,0.011818,(INDOMIE GORENG ACEH/40 IMGA),1
7,0.010692,(INDOMIE GRG JMBO A.PGG/24 GAPJ),1
8,0.014631,(INDOMIE GRG JUMBO/24 GSSJ),1
9,0.014631,(INDOMIE GRG RENDANG/40 GRS),1
